<a href="https://colab.research.google.com/github/jpsiegel/Projects/blob/master/Feudal_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib
from pylab import *
from IPython import display
import time
import numpy as np

In [0]:
# Parameters

vassal_grid_size = 4 # n x n
lord_grid_size = vassal_grid_size - 1
governor_grid_size = lord_grid_size - 1

initial_army_level = 1
initial_loyalty_value = 1
army_growth_rate = 0.1
loyalty_decay_rate = 0.01

In [0]:
# Classes


class Vassal:  # Vasallo

  id_ = 0

  def __init__(self, initial_army_level, initial_loyalties_up):
    self.id_ = Vassal.id_
    self.army_value = initial_army_level
    self.loyalties_up = initial_loyalties_up #dict: {L1: loyalty_value1, L2: l_v2}
    Vassal.id_ += 1

  def __repr__(self):
    return "Vassal{} loyalties_up:{}".format(self.id_, self.loyalties_up)


class Lord:  # Señor

  id_ = 0

  def __init__(self, initial_loyalties_up, initial_loyalties_down):
    self.id_ = Lord.id_
    self.loyalties_up = initial_loyalties_up  # dict: {G1: loyalty_value1, G2: l_v2}
    self.loyalties_down = initial_loyalties_down  # dict: {V1: loyalty_value1, V2: l_v2}
    Lord.id_ += 1
    
  def army_value(self):
    """Function to calculate army value of Lord given its loyalties down"""
    pass

  def __repr__(self):
    return "Lord{} loyalties_down:{}".format(self.id_, self.loyalties_down)


class Governor:  # Gobernador

  id_ = 0

  def __init__(self, initial_loyalties_down):
    self.id_ = Governor.id_
    self.loyalties_down = initial_loyalties_down  # dict: {L1: loyalty_value1, L2: l_v2}
    Governor.id_ += 1
    
  def army_value(self):
    """Function to calculate army value of Governor given its loyalties down"""
    pass

  def __repr__(self):
    return "Governor{} loyalties_down:{}".format(self.id_, self.loyalties_down)

In [0]:
#Auxiliar functions

def connect_down(A, B, value=initial_loyalty_value):
  """Connects A down to B. A will have a loyalties_down pointing to B.
  B will have a loyalties_up pointing to A"""
  A.loyalties_down[B.id_] = value
  B.loyalties_up[A.id_] = value

In [0]:
def initialize():
    global vassal_config, vassal_nextconfig, lord_config, lord_nextconfig, governor_config, governor_nextconfig

    #initialize vassal level
    vassal_config = np.zeros([vassal_grid_size, vassal_grid_size], dtype=type(Vassal(initial_army_level, {})))

    for x in range(vassal_grid_size):
      for y in range(vassal_grid_size):
        vassal_config[x, y] = Vassal(initial_army_level, {})

    vassal_nextconfig = np.zeros([vassal_grid_size, vassal_grid_size])

    #initialize lord level
    lord_config = np.zeros([lord_grid_size, lord_grid_size], dtype=type(Lord({}, {})))

    for x in range(lord_grid_size):
      for y in range(lord_grid_size):
        lord_config[x, y] = Lord({}, {})

    lord_nextconfig = np.zeros([lord_grid_size, lord_grid_size])

    #initialize governor level
    governor_config = np.zeros([governor_grid_size, governor_grid_size], dtype=type(Governor({})))

    for x in range(governor_grid_size):
      for y in range(governor_grid_size):
        governor_config[x, y] = Governor({})

    governor_nextconfig = np.zeros([governor_grid_size, governor_grid_size])

    #connect all levels
    for x in range(governor_grid_size):
      for y in range(governor_grid_size):
        connect_down(governor_config[x,y], lord_config[x, y])
        connect_down(governor_config[x,y], lord_config[x+1, y+1])

    for x in range(lord_grid_size):
      for y in range(lord_grid_size):
        connect_down(lord_config[x,y], vassal_config[x, y])
        connect_down(lord_config[x,y], vassal_config[x+1, y+1])

    print("Vassal Initial Config:")
    print(vassal_config)
    print("")
    print("Lord Initial Config:")
    print(lord_config)
    print("")
    print("Governor Initial Config:")
    print(governor_config)
    print("")
########################################################################################################

def observe(color_map="viridis"):
    global config, nextconfig
    cla()
    imshow(config, vmin = 0, vmax = 8, cmap = color_map)

def update():
    global config, nextconfig

    #iterate through grid
    for x in range(n):
        for y in range(n):
            count = 0
            central_value = config[x % n, y % n]

            if central_value != 0: #this cell is only excitable if it's in state 0
              #if it's not excitable, we just advance in state and skip neighbourhood check
              if central_value + 1 <= k:
                nextconfig[x, y] = central_value + 1
              else:
                nextconfig[x, y] = 0
              continue

            #cell is excitable, iterate through (x,y)'s moore neighbourhood
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    if dx == 0 and dy == 0: #dont count central cell (outer totalistic)
                      continue
                    # % n makes it cross frontiers
                    count += 1 if config[(x + dx) % n, (y + dy) % n] == 1 else 0  # we add to counter only if this neighbour is excited

            nextconfig[x, y] = 1 if np.random.uniform(0, 1) <= (count/4) else 0 #estochastic excitement condition
            #nextconfig[x, y] = 1 if count > 1 else 0 #deterministic excitement condition

    config, nextconfig = nextconfig, config

In [6]:
# Simulate and plot results
initialize()
#for t in range(1000):
#    update()
#    observe('twilight_shifted')
#    display.clear_output(wait=True)
#    display.display(gcf())
#    time.sleep(0.1)
#    plt.close()
#plt.show()

Vassal Initial Config:
[[Vassal1 loyalties_up:{1: 1} Vassal2 loyalties_up:{2: 1}
  Vassal3 loyalties_up:{3: 1} Vassal4 loyalties_up:{}]
 [Vassal5 loyalties_up:{4: 1} Vassal6 loyalties_up:{1: 1, 5: 1}
  Vassal7 loyalties_up:{2: 1, 6: 1} Vassal8 loyalties_up:{3: 1}]
 [Vassal9 loyalties_up:{7: 1} Vassal10 loyalties_up:{4: 1, 8: 1}
  Vassal11 loyalties_up:{5: 1, 9: 1} Vassal12 loyalties_up:{6: 1}]
 [Vassal13 loyalties_up:{} Vassal14 loyalties_up:{7: 1}
  Vassal15 loyalties_up:{8: 1} Vassal16 loyalties_up:{9: 1}]]

Lord Initial Config:
[[Lord1 loyalties_down:{1: 1, 6: 1} Lord2 loyalties_down:{2: 1, 7: 1}
  Lord3 loyalties_down:{3: 1, 8: 1}]
 [Lord4 loyalties_down:{5: 1, 10: 1} Lord5 loyalties_down:{6: 1, 11: 1}
  Lord6 loyalties_down:{7: 1, 12: 1}]
 [Lord7 loyalties_down:{9: 1, 14: 1} Lord8 loyalties_down:{10: 1, 15: 1}
  Lord9 loyalties_down:{11: 1, 16: 1}]]

Governor Initial Config:
[[Governor1 loyalties_down:{1: 1, 5: 1}
  Governor2 loyalties_down:{2: 1, 6: 1}]
 [Governor3 loyalties_down